## Reading Data

In [ ]:
# Import package
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [83]:
# Read data
df = pd.read_csv('data_interview.csv')

In [84]:
# Print head of df
df.head(4)

,STT,date_time,event_name,level,user,day_diff,day0,mode_game,win,reason_to_die,quantity,version
0,40,10/28/2023,game_start,1,1cffd052-4616-4d29-bfd5-950e23b763d4,0,10/28/2023,normal,NaN,NaN,0,1.6.0
1,41,10/28/2023,game_start,1,1cffd052-4616-4d29-bfd5-950e23b763d4,0,10/28/2023,normal,NaN,NaN,0,1.6.0
2,42,10/28/2023,game_start,2,1cffd052-4616-4d29-bfd5-950e23b763d4,0,10/28/2023,normal,NaN,NaN,0,1.6.0
3,43,10/28/2023,game_start,3,1cffd052-4616-4d29-bfd5-950e23b763d4,0,10/28/2023,normal,NaN,NaN,0,1.6.0


## EDA

In [85]:
# Check df info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798332 entries, 0 to 798331
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   STT            798332 non-null  int64  
 1   date_time      798332 non-null  object 
 2   event_name     798332 non-null  object 
 3   level          798332 non-null  int64  
 4   user           798332 non-null  object 
 5   day_diff       798332 non-null  int64  
 6   day0           798332 non-null  object 
 7   mode_game      798332 non-null  object 
 8   win            243697 non-null  float64
 9   reason_to_die  117413 non-null  object 
 10  quantity       798332 non-null  int64  
 11  version        798332 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 73.1+ MB


In [86]:
# Check value of win column
df['win'].value_counts()

win
1.0    126284
0.0    117413
Name: count, dtype: int64

In [87]:
# Check value of reason_to_die column
df['reason_to_die'].value_counts()

reason_to_die
out_of_lives         99966
out_of_lives_plus    17447
Name: count, dtype: int64

In [88]:
# Replace NaN values with a blank string in win, reaseon_to_die columns:
df['win'] = df['win'].fillna('')
df['reason_to_die'] = df['reason_to_die'].fillna('')

In [89]:
df['win'].value_counts()

win
       554635
1.0    126284
0.0    117413
Name: count, dtype: int64

In [90]:
# Check how many value mode game column is tutorial
df['mode_game'].value_counts()

mode_game
normal      493496
ue          159896
tutorial    108587
ss           36353
Name: count, dtype: int64

In [91]:
# Check value of quantity column with filter tutorial in mode game column
df[df['mode_game'] == 'tutorial']['quantity'].value_counts()


quantity
-1    14212
 1    13543
 2    13384
 3    13317
 4    13267
-2    13209
 5     6749
 6     6736
 7     6726
 8     6713
 0      731
Name: count, dtype: int64

In [92]:
# The number of user skip the tutorial between two version
df[(df['mode_game'] == 'tutorial') & (df['quantity'] == 0)]['version'].value_counts()

version
1.6.0    372
1.5.2    359
Name: count, dtype: int64

In [93]:
# How many steps in tutorial for  version 1.6.0
df[(df['mode_game'] == 'tutorial') & (df['version'] == '1.6.0')]['quantity'].value_counts()

quantity
-1    7224
 1    6885
 2    6811
 3    6782
 4    6759
 5    6749
 6    6736
 7    6726
 8    6713
-2    6712
 0     372
Name: count, dtype: int64

In [94]:
# How many steps in tutorial for  version 1.5.2
df[(df['mode_game'] == 'tutorial') & (df['version'] == '1.5.2')]['quantity'].value_counts()

quantity
-1    6988
 1    6658
 2    6573
 3    6535
 4    6508
-2    6497
 0     359
Name: count, dtype: int64

In [138]:
# day_diff vs tutorial
df[(df['mode_game'] == 'tutorial')]['day_diff'].value_counts()

day_diff
0    107161
1       817
2       252
3       144
6        66
4        62
7        56
5        29
Name: count, dtype: int64

In [139]:
df['user'].nunique()

13584

In [141]:
df[df['event_name'] == 'user_engagement']['version'].value_counts()

version
1.6.0    106113
1.5.2     90136
Name: count, dtype: int64

In [ ]:
# Conlusion
# No much diffrent in number of user completed the tutorial, skip the tutorial.
# There are 4 steps in tutorial for version 1.5.2, 8 steps for version 1.6.0.
# Most of people watch the tutorial in the same day they openned the game.
# There are 13584 unique user in data
# User engagement in version 1.6.0 is more than in version 1.5.2

## Divine data into two group of version 1.5.2, 1.6.0

In [142]:
# Filter data for group version 1.5.2
df_152 = df[df['version'] == '1.5.2']
print(df_152.shape)

(357718, 12)


In [143]:
# Filter data for group version 1.6.0
df_160 = df[df['version'] == '1.6.0']
print(df_160.shape)

(440614, 12)


## Calculating retention

In [146]:
# The purpose of this function : Detect status of tutorial based on quantity column
def map_tutorial_status(row):
    if row['event_name'] == 'tutorial':
        # Apply the original function logic when 'e' column is 'tutorial'
        if row['quantity'] == -2:
            return "completed"
        elif row['quantity'] == 0:
            return "skip"
        elif row['quantity'] in (-1, 1, 2, 3, 4, 5, 6, 7, 8):
            return "processing"
    # Otherwise, return "duration" for other cases
    return "duration"

In [147]:
# Apply the map_tutorial_status function for version 1.5.2
df_152['tutorial_status'] = df_152.apply(map_tutorial_status, axis=1)


C:\Users\USER\AppData\Local\Temp\ipykernel_27580\3281214792.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_152['tutorial_status'] = df_152.apply(map_tutorial_status, axis=1)


In [ ]:
# NUmber of user completed the tutorial for version 1.5.2
# Filter rows where tutorial_status is "completed"
completed_152_df = df_152[df_152['tutorial_status'] == 'completed']

# Group by user_id and count occurrences
completed_152_counts = completed_152_df.groupby('user').size().reset_index(name='completed_152_counts')

# Count number unique user completed the tutorial
completed_152_counts['completed_152_counts'].sum()

In [ ]:
# Filter user who had completed the tutorial
unique_user_152 = completed_152_counts['user'].tolist()

In [ ]:
# NUmber of use who return after completed the tutorial for version 1.5.2
returned_152_df = df_152[(df_152['day_diff'] > 0) & (df_152['user'].isin(unique_user_152))]

# Group by user_id and count occurrences
returned_152_counts = returned_152_df.groupby('user')['date_time'].nunique().reset_index(name='returned_152_counts')

# Count number unique user who returns after completing tutorial
returned_152_counts['user'].count()

In [193]:
# Retention rate for version 1.5.2
retension_rate_152 = 2335/6497

In [172]:
# Apply the map_tutorial_status func for version 1.6.0
df_160['tutorial_status'] = df_160.apply(map_tutorial_status, axis=1)


C:\Users\USER\AppData\Local\Temp\ipykernel_27580\3482044717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_160['tutorial_status'] = df_160.apply(map_tutorial_status, axis=1)


In [174]:
# NUmber of user completed the tutorial for version 1.6.0
# Filter rows where tutorial_status is "completed"
completed_160_df = df_160[df_160['tutorial_status'] == 'completed']

# Group by user_id and count occurrences
completed_counts_160 = completed_160_df.groupby('user').size().reset_index(name='completed_counts_160')

# Count number of user who completed the tutorial
completed_counts_160['completed_counts_160'].sum()

np.int64(6712)

In [175]:
# Filter user who had completed the tutorial
unique_user_160 = completed_counts_160['user'].tolist()

In [176]:
# NUmber of use who return after completed the tutorial for version 1.6.0
returned_df = df_160[(df_160['day_diff'] > 0) & (df_160['user'].isin(unique_user_160))]

# Group by user_id and count occurrences
returned_counts_160 = returned_df.groupby('user')['date_time'].nunique().reset_index(name='returned_count_160')

# NUmber of user who returned after watching the tutorial
returned_counts_160['user'].count()


np.int64(2727)

In [194]:
# Retention rate for version 1.6.0
retention_rate_160 = 2727/6712

## Win rate after completing the tutorial

In [188]:
# Version 1.5.2
win_unique_user_152 = df_152[(df_152['day_diff'] > 0) & (df_152['user'].isin(unique_user_152)) & (df_152['win'] == 1.0)]['user'].nunique()

In [189]:
# Win rate for version 1.5.2 : win_unique_user / total unique user completed the tutorial

win_unique_user_152 / 6497

0.29167307988302293

In [190]:
# Version 1.6.0
win_unique_user_160 = df_160[(df_160['day_diff'] > 0) & (df_160['user'].isin(unique_user_160)) & (df_160['win'] == 1.0)]['user'].nunique()

In [191]:
# Win rate for version 1.6.0 : win_unique_user / total unique user completed the tutorial

win_unique_user_160 / 6712

0.33134684147794996

## a/b testing Win rate

In [204]:
from statsmodels.stats.proportion import proportions_ztest

# Successes and observations for each group
successes = [win_unique_user_152, win_unique_user_160]
observations = [6497, 6712]

# Perform Z-Test
z_stat, p_val = proportions_ztest(successes, observations)
print(f"Z-Statistic: {z_stat}, P-Value: {p_val}")

if p_val < 0.05:
    print("Statistically significant difference in win rates between Version 1.5.2 and Version 1.6.0.")
else:
    print("No statistically significant difference in win rates between Version 1.5.2 and Version 1.6.0.")

Z-Statistic: -4.920881637805303, P-Value: 8.615522485345721e-07
Statistically significant difference in win rates between Version 1.5.2 and Version 1.6.0.


In [216]:
# Successes and observations for each group
successes = [retension_rate_152, retention_rate_160]
observations = [6497, 6712]

# Perform Z-Test
z_stat, p_val = proportions_ztest(successes, observations)
print(f"Z-Statistic: {z_stat}, P-Value: {p_val}")

if p_val < 0.05:
    print("Statistically significant difference in retention rates between Version 1.5.2 and Version 1.6.0.")
else:
    print("No statistically significant difference in retention rates between Version 1.5.2 and Version 1.6.0.")

Z-Statistic: -0.03935082096821081, P-Value: 0.9686106886941962
No statistically significant difference in retention rates between Version 1.5.2 and Version 1.6.0.


In [218]:
df_152.sort_values(by = ['date_time'], ascending=False)

,STT,date_time,event_name,level,user,day_diff,day0,mode_game,win,reason_to_die,quantity,version,tutorial_status
723575,4310996,11/9/2023,game_end,62,98d9de28-c32e-4994-bbf0-8ebc4ccfaaad,6,11/3/2023,normal,1.0,,211,1.5.2,duration
749037,4525265,11/9/2023,user_engagement,18,125a062c-ec99-43e4-9af9-3687b57f1b90,6,11/3/2023,ue,,,0,1.5.2,duration
748413,4521693,11/9/2023,user_engagement,32,87f14937-2eb9-43b8-a21f-3e42bdc8032b,6,11/3/2023,ue,,,0,1.5.2,duration
748414,4521694,11/9/2023,user_engagement,32,87f14937-2eb9-43b8-a21f-3e42bdc8032b,6,11/3/2023,ue,,,0,1.5.2,duration
748415,4521695,11/9/2023,user_engagement,32,87f14937-2eb9-43b8-a21f-3e42bdc8032b,6,11/3/2023,ue,,,0,1.5.2,duration
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73141,481235,10/28/2023,user_engagement,1,ee9044fc-adb4-4b34-9c04-26a46e63b6d1,0,10/28/2023,ss,,,0,1.5.2,duration
73140,481233,10/28/2023,tutorial,1,ee9044fc-adb4-4b34-9c04-26a46e63b6d1,0,10/28/2023,tutorial,,,4,1.5.2,processing
73139,481232,10/28/2023,tutorial,1,ee9044fc-adb4-4b34-9c04-26a46e63b6d1,0,10/28/2023,tutorial,,,3,1.5.2,processing
73138,481231,10/28/2023,tutorial,1,ee9044fc-adb4-4b34-9c04-26a46e63b6d1,0,10/28/2023,tutorial,,,2,1.5.2,processing
